In [0]:
%sql
-- create table using DELTA

create or replace table orders(
  order_id INT,
  order_date STRING,
  order_item_id INT,
  order_status STRING
) USING DELTA;

-- you can use PARQUET,CSV,JSON as well

We are going to first drop dn and then create DB

In [0]:
%sql
DROP DATABASE IF EXISTS itversity_retail_db CASCADE;

Set the data warehouse location 
By default it is dbfs:/user/hive/warehouse

In [0]:
%sql
SET spark.sql.warehouse.dir;

--OP: spark.sql.warehouse.dir	*********(redacted)

key,value
spark.sql.warehouse.dir,*********(redacted)


In [0]:
%sql
DROP DATABASE IF EXISTS itversity_retail_db;

In [0]:

dbutils.fs.mkdirs("dbfs:/public/warehouse")


True

In [0]:
%fs ls dbfs:/public

path,name,size,modificationTime
dbfs:/public/retail_db/,retail_db/,0,1713775512800
dbfs:/public/retail_db_parquet/,retail_db_parquet/,0,0
dbfs:/public/schema_for_retail_db.jpg,schema_for_retail_db.jpg,54885,1713956587559
dbfs:/public/warehouse/,warehouse/,0,1714041445347


In [0]:
%sql
CREATE DATABASE IF NOT EXISTS itversity_retail_db_1;


--SHOW SCHEMAS;

In [0]:
%sql
SHOW DATABASES

databaseName
default
information_schema
itversity_retail_db_1


In [0]:
%sql
use itversity_retail_db_1

In [0]:
%sql
select current_database()

current_database()
itversity_retail_db_1


In [0]:
%sql
select current_catalog()

current_catalog()
sovik_analytics


In [0]:
%sql
create or replace TABLE itversity_retail_db_1.orders(
  order_id INT,
  order_date DATE,
  order_item_id INT,
  order_status STRING
) USING DELTA;

In [0]:
%sql
SHOW TABLES;

database,tableName,isTemporary
itversity_retail_db_1,orders,false


In [0]:
%sql
--with extra values
DESCRIBE FORMATTED orders;

col_name,data_type,comment
order_id,int,null
order_date,date,null
order_item_id,int,null
order_status,string,null
,,
# Detailed Table Information,,
Catalog,sovik_analytics,
Database,itversity_retail_db_1,
Table,orders,
Created Time,Thu Apr 25 12:32:33 UTC 2024,


col_name	data_type
order_id	int
order_date	date
order_item_id	int
order_status	string
	
# Detailed Table Information	
Catalog	sovik_analytics
Database	itversity_retail_db_1
Table	orders
Created Time	Thu Apr 25 12:32:33 UTC 2024
Last Access	UNKNOWN
Created By	Spark 
Type	MANAGED
Location	gs://databricks-2134317533702172-unitycatalog/2134317533702172/__unitystorage/catalogs/9b4c1bfb-b27f-4fa2-99cc-02193af99209/tables/3cc9490a-58a4-4677-8a20-1cb2789b47a6
Provider	delta
Owner	sovikstudy2024@gmail.com
Is_managed_location	true
Table Properties	[delta.minReaderVersion=1,delta.minWriterVersion=2]

In [0]:
%sql

--showing the table with proper BTS of the creation
show create table orders;

createtab_stmt
"CREATE TABLE sovik_analytics.default.orders ( order_id INT, order_date STRING, order_item_id INT, order_status STRING) USING delta TBLPROPERTIES ( 'delta.minReaderVersion' = '1', 'delta.minWriterVersion' = '2')"


In [0]:
%fs ls dbfs:/

path,name,size,modificationTime
dbfs:/FileStore/,FileStore/,0,0
dbfs:/Volume/,Volume/,0,0
dbfs:/Volumes/,Volumes/,0,0
dbfs:/databricks-datasets/,databricks-datasets/,0,0
dbfs:/databricks-results/,databricks-results/,0,0
dbfs:/public/,public/,0,1714048735241
dbfs:/volume/,volume/,0,0
dbfs:/volumes/,volumes/,0,0


In [0]:
dbutils.fs.mkdirs("dbfs:/public/retail_db_json")

True

In [0]:
%fs ls dbfs:/public/retail_db_json/orders


path,name,size,modificationTime
dbfs:/public/retail_db_json/orders/part-00000,part-00000,7477339,1714129058797


In [0]:
%sql
describe table orders

col_name,data_type,comment
order_id,int,null
order_date,string,null
order_item_id,int,null
order_status,string,null


In [0]:
%sql
COPY INTO orders
FROM 'dbfs:/public/retail_db_json/orders'
FILEFORMAT = JSON
FORMAT_OPTIONS ('inferSchema' = 'true')
COPY_OPTIONS ('mergeSchema' = 'true');

com.databricks.backend.common.rpc.SparkDriverExceptions$SQLExecutionException: com.databricks.sql.transaction.tahoe.DeltaAnalysisException: Failed to merge fields 'order_id' and 'order_id'
	at com.databricks.sql.transaction.tahoe.schema.SchemaMergingUtils$.$anonfun$mergeSchemas$1(SchemaMergingUtils.scala:204)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:286)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:198)
	at scala.collection.TraversableLike.map(TraversableLike.scala:286)
	at scala.collection.TraversableLike.map$(TraversableLike.scala:279)
	at scala.collection.mutable.ArrayOps$ofRef.map(ArrayOps.scala:198)
	at com.databricks.sql.transaction.tahoe.schema.SchemaMergingUtils$.merge$1(SchemaMergingUtils.scala:184)
	at com.databricks.sql.transaction.tahoe.schema.SchemaMergingUtils$.mergeSchemas(SchemaMergingUtils.scala:286)
	at com.databricks.sql.transaction.tahoe.schema.ImplicitMetadataOperation$.mergeSchema(ImplicitMetadataOperation.scala:229)
	at com.databricks.sql.transaction.tahoe.commands.copy.CopyIntoStatementEdge.buildCopyIntoOptions(CopyIntoCommandEdge.scala:270)
	at com.databricks.sql.transaction.tahoe.commands.copy.CopyIntoStatementEdge.$anonfun$toCopyIntoCommand$2(CopyIntoCommandEdge.scala:201)
	at org.apache.spark.sql.catalyst.MetricKeyUtils$.measureMs(MetricKey.scala:520)
	at com.databricks.sql.transaction.tahoe.commands.copy.CopyIntoStatementEdge$.$anonfun$recordDuration$1(CopyIntoCommandEdge.scala:433)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:94)
	at com.databricks.sql.transaction.tahoe.commands.copy.CopyIntoStatementEdge$.recordDuration(CopyIntoCommandEdge.scala:432)
	at com.databricks.sql.transaction.tahoe.commands.copy.CopyIntoStatementEdge.toCopyIntoCommand(CopyIntoCommandEdge.scala:178)
	at com.databricks.sql.transaction.tahoe.DeltaAnalysis$$anonfun$apply$1.$anonfun$applyOrElse$33(DeltaAnalysis.scala:940)
	at org.apache.spark.sql.catalyst.MetricKeyUtils$.measureMs(MetricKey.scala:520)
	at com.databricks.sql.transaction.tahoe.commands.copy.CopyIntoStatementEdge$.$anonfun$recordDuration$1(CopyIntoCommandEdge.scala:433)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:94)
	at com.databricks.sql.transaction.tahoe.commands.copy.CopyIntoStatementEdge$.recordDuration(CopyIntoCommandEdge.scala:432)
	at com.databricks.sql.transaction.tahoe.DeltaAnalysis$$anonfun$apply$1.applyOrElse(DeltaAnalysis.scala:890)
	at com.databricks.sql.transaction.tahoe.DeltaAnalysis$$anonfun$apply$1.applyOrElse(DeltaAnalysis.scala:106)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDownWithPruning$2(AnalysisHelper.scala:219)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(origin.scala:69)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDownWithPruning$1(AnalysisHelper.scala:219)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:395)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDownWithPruning(AnalysisHelper.scala:217)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDownWithPruning$(AnalysisHelper.scala:213)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsDownWithPruning(LogicalPlan.scala:39)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDown(AnalysisHelper.scala:209)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDown$(AnalysisHelper.scala:208)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsDown(LogicalPlan.scala:39)
	at com.databricks.sql.transaction.tahoe.DeltaAnalysis.apply(DeltaAnalysis.scala:106)
	at com.databricks.sql.transacti

DEBUGGING

In [0]:


source_df = spark.read.json("dbfs:/public/retail_db_json/orders")
source_df.printSchema()


root
 |-- order_customer_id: long (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_id: long (nullable = true)
 |-- order_status: string (nullable = true)



In [0]:
%sql
DESCRIBE table orders

col_name,data_type,comment
order_id,int,null
order_date,string,null
order_item_id,int,null
order_status,string,null


In [0]:
from delta.tables import DeltaTable

delta_table = DeltaTable.forName(spark, "orders")
delta_table.toDF().printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- order_status: string (nullable = true)



In [0]:
from pyspark.sql.functions import col

source_df = source_df.withColumn("order_id", col("order_id").cast("integer"))
source_df = source_df.withColumn("order_item_id", col("order_id").cast("integer"))
source_df.printSchema()

root
 |-- order_customer_id: long (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_id: integer (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_item_id: integer (nullable = true)



In [0]:
source_df.show()

+-----------------+--------------------+--------+---------------+-------------+
|order_customer_id|          order_date|order_id|   order_status|order_item_id|
+-----------------+--------------------+--------+---------------+-------------+
|            11599|2013-07-25 00:00:...|       1|         CLOSED|            1|
|              256|2013-07-25 00:00:...|       2|PENDING_PAYMENT|            2|
|            12111|2013-07-25 00:00:...|       3|       COMPLETE|            3|
|             8827|2013-07-25 00:00:...|       4|         CLOSED|            4|
|            11318|2013-07-25 00:00:...|       5|       COMPLETE|            5|
|             7130|2013-07-25 00:00:...|       6|       COMPLETE|            6|
|             4530|2013-07-25 00:00:...|       7|       COMPLETE|            7|
|             2911|2013-07-25 00:00:...|       8|     PROCESSING|            8|
|             5657|2013-07-25 00:00:...|       9|PENDING_PAYMENT|            9|
|             5648|2013-07-25 00:00:...|

In [0]:
%sql
create or replace table orders_v2(
  order_customer_id BIGINT,  
  order_date STRING,
  order_id BIGINT,  
  order_status STRING,
  order_item_id BIGINT
) USING DELTA;

In [0]:
%sql
DESCRIBE table orders_v2

col_name,data_type,comment
order_customer_id,bigint,null
order_date,string,null
order_id,bigint,null
order_status,string,null
order_item_id,bigint,null


In [0]:
%sql
COPY INTO orders_v2
FROM "dbfs:/public/retail_db_json/orders"
FILEFORMAT=JSON;


num_affected_rows,num_inserted_rows,num_skipped_corrupt_files
68883,68883,0


In [0]:
%sql
select * from orders_v2 LIMIT 20


order_customer_id,order_date,order_id,order_status,order_item_id
11599,2013-07-25 00:00:00.0,1,CLOSED,null
256,2013-07-25 00:00:00.0,2,PENDING_PAYMENT,null
12111,2013-07-25 00:00:00.0,3,COMPLETE,null
8827,2013-07-25 00:00:00.0,4,CLOSED,null
11318,2013-07-25 00:00:00.0,5,COMPLETE,null
7130,2013-07-25 00:00:00.0,6,COMPLETE,null
4530,2013-07-25 00:00:00.0,7,COMPLETE,null
2911,2013-07-25 00:00:00.0,8,PROCESSING,null
5657,2013-07-25 00:00:00.0,9,PENDING_PAYMENT,null
5648,2013-07-25 00:00:00.0,10,PENDING_PAYMENT,null


In [0]:
%sql
select count(*) from orders_v2; 

count(1)
68883


create external table

In [0]:
source_df = spark.read.json("dbfs:/public/retail_db_json/order_items")
source_df.printSchema()

root
 |-- order_item_id: long (nullable = true)
 |-- order_item_order_id: long (nullable = true)
 |-- order_item_product_id: long (nullable = true)
 |-- order_item_product_price: double (nullable = true)
 |-- order_item_quantity: long (nullable = true)
 |-- order_item_subtotal: double (nullable = true)



In [0]:
%sql
use default

In [0]:
%sql
CREATE EXTERNAL TABLE order_items (
  order_item_id BIGINT,
  order_item_order_id BIGINT,
  order_item_product_id BIGINT,
  order_item_product_price DOUBLE,
  order_item_quantity BIGINT,
  order_item_subtotal DOUBLE
)
USING DELTA
LOCATION 'dbfs:/public/warehouse/';

com.databricks.backend.common.rpc.SparkDriverExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: [UC_FILE_SCHEME_FOR_TABLE_CREATION_NOT_SUPPORTED] Creating table in Unity Catalog with file scheme dbfs is not supported.
Instead, please create a federated data source connection using the CREATE CONNECTION command for the same table provider, then create a catalog based on the connection with a CREATE FOREIGN CATALOG command to reference the tables therein.
	at com.databricks.unity.CredentialScopeSQLHelper$.checkPathOperations(CredentialScopeSQLHelper.scala:94)
	at com.databricks.unity.CredentialScopeSQLHelper$.register(CredentialScopeSQLHelper.scala:145)
	at com.databricks.unity.CredentialScopeSQLHelper$.registerCreateTableAccess(CredentialScopeSQLHelper.scala:592)
	at com.databricks.sql.managedcatalog.CredentialScopeTableCredentialHandler.injectCredential(ResolveWithCredential.scala:480)
	at com.databricks.sql.managedcatalog.ResolveWithCredential.com$databricks$sql$managedcatalog$ResolveWithCredential$$maybeDecorateDSv2Table(ResolveWithCredential.scala:134)
	at com.databricks.sql.managedcatalog.ResolveWithCredential$$anonfun$apply$1.applyOrElse(ResolveWithCredential.scala:166)
	at com.databricks.sql.managedcatalog.ResolveWithCredential$$anonfun$apply$1.applyOrElse(ResolveWithCredential.scala:147)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDownWithPruning$2(AnalysisHelper.scala:219)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(origin.scala:69)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDownWithPruning$1(AnalysisHelper.scala:219)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:395)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDownWithPruning(AnalysisHelper.scala:217)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDownWithPruning$(AnalysisHelper.scala:213)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsDownWithPruning(LogicalPlan.scala:39)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDown(AnalysisHelper.scala:209)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDown$(AnalysisHelper.scala:208)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsDown(LogicalPlan.scala:39)
	at com.databricks.sql.managedcatalog.ResolveWithCredential.apply(ResolveWithCredential.scala:147)
	at com.databricks.sql.managedcatalog.ResolveWithCredential.apply(ResolveWithCredential.scala:62)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$4(RuleExecutor.scala:286)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:94)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$3(RuleExecutor.scala:286)
	at scala.collection.LinearSeqOptimized.foldLeft(LinearSeqOptimized.scala:126)
	at scala.collection.LinearSeqOptimized.foldLeft$(LinearSeqOptimized.scala:122)
	at scala.collection.immutable.List.foldLeft(List.scala:91)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$2(RuleExecutor.scala:283)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:94)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.executeBatch$1(RuleExecutor.scala:266)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$8(RuleExecutor.scala:353)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$8$adapted(RuleExecutor.scala:353)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$1(RuleExecutor.scala:353)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:94)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.e

AnalysisException: [UC_FILE_SCHEME_FOR_TABLE_CREATION_NOT_SUPPORTED] Creating table in Unity Catalog with file scheme dbfs is not supported.
Instead, please create a federated data source connection using the CREATE CONNECTION command for the same table provider, then create a catalog based on the connection with a CREATE FOREIGN CATALOG command to reference the tables therein.

To resolve this issue, you can follow the steps below:

Create a Federated Data Source Connection:
Use the CREATE CONNECTION command to create a connection to the data source that you want to reference.
For a Delta Lake file path, you would typically create a connection using the DELTA_LAKE provider.
The connection will specify the location of the data (e.g., DBFS path) and any necessary connection details.
Here’s an example of how you might create a connection to a Delta Lake data source:

In [0]:
%sql
CREATE EXTERNAL TABLE my_catalog.order_items (
  order_item_id BIGINT,
  order_item_order_id BIGINT,
  order_item_product_id BIGINT,
  order_item_product_price DOUBLE,
  order_item_quantity BIGINT,
  order_item_subtotal DOUBLE
)
USING DELTA
LOCATION 'dbfs:/public/warehouse/itversity_retail_db_1.db/order_items';

com.databricks.backend.common.rpc.SparkDriverExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: [UC_FILE_SCHEME_FOR_TABLE_CREATION_NOT_SUPPORTED] Creating table in Unity Catalog with file scheme dbfs is not supported.
Instead, please create a federated data source connection using the CREATE CONNECTION command for the same table provider, then create a catalog based on the connection with a CREATE FOREIGN CATALOG command to reference the tables therein.
	at com.databricks.unity.CredentialScopeSQLHelper$.checkPathOperations(CredentialScopeSQLHelper.scala:94)
	at com.databricks.unity.CredentialScopeSQLHelper$.register(CredentialScopeSQLHelper.scala:145)
	at com.databricks.unity.CredentialScopeSQLHelper$.registerCreateTableAccess(CredentialScopeSQLHelper.scala:592)
	at com.databricks.sql.managedcatalog.CredentialScopeTableCredentialHandler.injectCredential(ResolveWithCredential.scala:480)
	at com.databricks.sql.managedcatalog.ResolveWithCredential.com$databricks$sql$managedcatalog$ResolveWithCredential$$maybeDecorateDSv2Table(ResolveWithCredential.scala:134)
	at com.databricks.sql.managedcatalog.ResolveWithCredential$$anonfun$apply$1.applyOrElse(ResolveWithCredential.scala:166)
	at com.databricks.sql.managedcatalog.ResolveWithCredential$$anonfun$apply$1.applyOrElse(ResolveWithCredential.scala:147)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDownWithPruning$2(AnalysisHelper.scala:219)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(origin.scala:69)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDownWithPruning$1(AnalysisHelper.scala:219)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:395)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDownWithPruning(AnalysisHelper.scala:217)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDownWithPruning$(AnalysisHelper.scala:213)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsDownWithPruning(LogicalPlan.scala:39)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDown(AnalysisHelper.scala:209)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsDown$(AnalysisHelper.scala:208)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsDown(LogicalPlan.scala:39)
	at com.databricks.sql.managedcatalog.ResolveWithCredential.apply(ResolveWithCredential.scala:147)
	at com.databricks.sql.managedcatalog.ResolveWithCredential.apply(ResolveWithCredential.scala:62)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$4(RuleExecutor.scala:286)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:94)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$3(RuleExecutor.scala:286)
	at scala.collection.LinearSeqOptimized.foldLeft(LinearSeqOptimized.scala:126)
	at scala.collection.LinearSeqOptimized.foldLeft$(LinearSeqOptimized.scala:122)
	at scala.collection.immutable.List.foldLeft(List.scala:91)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$2(RuleExecutor.scala:283)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:94)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.executeBatch$1(RuleExecutor.scala:266)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$8(RuleExecutor.scala:353)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$8$adapted(RuleExecutor.scala:353)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$1(RuleExecutor.scala:353)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:94)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.e

USE itversity_retail_db
     

CREATE EXTERNAL TABLE order_items ( -- EXTERNAL is optional
  order_item_id BIGINT,
  order_item_order_id BIGINT,
  order_item_product_id BIGINT,
  order_item_quantity BIGINT,
  order_item_subtotal DOUBLE,
  order_item_product_price DOUBLE
) USING DELTA
OPTIONS (
  path='dbfs:/public/warehouse/itversity_retail_db.db/order_items' 
  -- If location is specified using path, then the table will be considered as external
)
     

%fs ls dbfs:/public/warehouse/itversity_retail_db.db
     

-- COPY INTO order_items
-- FROM 'dbfs:/public/retail_db_json/order_items'
-- FILEFORMAT = JSON

-- Used this for the demo\n
INSERT INTO order_items
SELECT order_item_id,
  order_item_order_id,
  order_item_product_id,
  order_item_quantity,
  order_item_subtotal,
  order_item_product_price
FROM JSON.`dbfs:/public/retail_db_json/order_items`


VALIDATE

USE itversity_retail_db
     

SELECT * FROM order_items
     

SELECT count(*) FROM order_items
     

SELECT count(*) FROM JSON.`dbfs:/public/retail_db_json/order_items
     

SELECT order_item_order_id, 
    round(sum(order_item_subtotal), 2) AS order_revenue
FROM order_items
GROUP BY 1
ORDER BY 1

In [0]:
%sql
In recent times external table structure is as follows:

CREATE TABLE <catalog>.<schema>.<table-name>
(
  <column-specification>
)
LOCATION 's3://<bucket-path>/<table-directory>';

In [0]:
%sql
use default

For CRUD OPS

In [0]:
%sql
CREATE  TABLE order_items (
  order_item_id BIGINT,
  order_item_order_id BIGINT,
  order_item_product_id BIGINT,
  order_item_product_price DOUBLE,
  order_item_quantity BIGINT,
  order_item_subtotal DOUBLE
)
USING DELTA

In [0]:
source_df = spark.read.json("dbfs:/public/retail_db_json/order_items")

source_df.show()

+-------------+-------------------+---------------------+------------------------+-------------------+-------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_product_price|order_item_quantity|order_item_subtotal|
+-------------+-------------------+---------------------+------------------------+-------------------+-------------------+
|            1|                  1|                  957|                  299.98|                  1|             299.98|
|            2|                  2|                 1073|                  199.99|                  1|             199.99|
|            3|                  2|                  502|                    50.0|                  5|              250.0|
|            4|                  2|                  403|                  129.99|                  1|             129.99|
|            5|                  4|                  897|                   24.99|                  2|              49.98|
|            6| 

In [0]:
%sql
COPY INTO order_items
FROM "dbfs:/public/retail_db_json/order_items"
FILEFORMAT=JSON;

num_affected_rows,num_inserted_rows,num_skipped_corrupt_files
172198,172198,0


In [0]:
%sql
select * from order_items LIMIT 30

order_item_id,order_item_order_id,order_item_product_id,order_item_product_price,order_item_quantity,order_item_subtotal
1,1,957,299.98,1,299.98
2,2,1073,199.99,1,199.99
3,2,502,50.0,5,250.0
4,2,403,129.99,1,129.99
5,4,897,24.99,2,49.98
6,4,365,59.99,5,299.95
7,4,502,50.0,3,150.0
8,4,1014,49.98,4,199.92
9,5,957,299.98,1,299.98
10,5,365,59.99,5,299.95


CRUD operations

In [0]:
%sql

-- CRUD
CREATE TABLE crud_demo (
    user_id INT,
    user_fname STRING,
    user_lname STRING,
    user_email STRING
) USING DELTA

In [0]:
%sql
INSERT INTO crud_demo 
VALUES
    (1, 'Scott', 'Tiger', 'stiger@email.com'),
    (2, 'Donald', 'Duck', 'dduck@email.com')

num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql
SELECT * FROM crud_demo

user_id,user_fname,user_lname,user_email
1,Mickey,Mouse,mmouse@email.com
2,Tom,Jerry,null
1,Scott,Tiger,stiger@email.com
2,Donald,Duck,dduck@email.com


In [0]:
%sql
INSERT INTO crud_demo 
VALUES
    (1, 'Mickey', 'Mouse', 'mmouse@email.com'),
    (2, 'Tom', 'Jerry', null)

num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql
DELETE FROM crud_demo
WHERE user_email IS NULL or user_email = 'mmouse@email.com'

num_affected_rows
2


In [0]:
%sql
SELECT * FROM crud_demo

user_id,user_fname,user_lname,user_email
1,Scott,Tiger,stiger@email.com
2,Donald,Duck,dduck@email.com


In [0]:
%sql
UPDATE  crud_demo SET user_lname='Sinclair' where user_email like 'stiger%';

num_affected_rows
1


In [0]:
%sql
SELECT * FROM crud_demo

user_id,user_fname,user_lname,user_email
1,Scott,Sinclair,stiger@email.com
2,Donald,Duck,dduck@email.com


In [0]:
%sql
INSERT INTO crud_demo 
VALUES
    (3, 'Mickey', 'Mouse', 'mmouse@email.com'),
    (4, 'Tom', 'Jerry', 'tjerry@gmail.com')

num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql
SELECT * FROM crud_demo

user_id,user_fname,user_lname,user_email
1,Scott,Sinclair,stiger@email.com
2,Donald,Duck,dduck@email.com
3,Mickey,Mouse,mmouse@email.com
4,Tom,Jerry,tjerry@gmail.com


MERGE ie insert or update

In [0]:
%sql
-- create staging table

CREATE TABLE crud_demo_stg (
    user_id INT,
    user_fname STRING,
    user_lname STRING,
    user_email STRING
) USING DELTA

In [0]:
%sql
INSERT INTO crud_demo_stg 
VALUES
    (3, 'Saurabh', 'Das', 'sdas@email.com'),
    (5, 'Lokesh', 'Rahul', 'lrahul@gmail.com'),
    (6, 'Jon', 'De la Claire', 'jondlc@gmail.com')

num_affected_rows,num_inserted_rows
3,3


In [0]:
%sql
merge into crud_demo as cd 
    using crud_demo_stg as cdg
    on cd.user_id=cdg.user_id
    when matched then update set *
    when not matched then insert *;


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
3,1,0,2


Explicitly specify columns

In [0]:
%sql
-- Explicitly specified columns
MERGE INTO crud_demo AS cd
    USING crud_demo_stg AS cdg
        ON cd.user_id = cdg.user_id
WHEN MATCHED THEN UPDATE SET
    cd.user_fname = cdg.user_fname,
    cd.user_lname = cdg.user_lname,
    cd.user_email = cdg.user_email
WHEN NOT MATCHED THEN INSERT
  (cd.user_id, cd.user_fname, cd.user_lname, cd.user_email)
VALUES
  (cdg.user_id, cdg.user_fname, cdg.user_lname, cdg.user_email)

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
3,3,0,0


In [0]:
%sql
select * from crud_demo

user_id,user_fname,user_lname,user_email
3,Saurabh,Das,sdas@email.com
5,Lokesh,Rahul,lrahul@gmail.com
6,Jon,De la Claire,jondlc@gmail.com
1,Scott,Sinclair,stiger@email.com
2,Donald,Duck,dduck@email.com
4,Tom,Jerry,tjerry@gmail.com
